In [1]:
%matplotlib notebook
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
from IPython.core.display import display, HTML
import numpy as np
import matplotlib.pyplot as plt

import importlib as imp
%cd ../

/media/work/docs/codes/c++/QSVT_framework/simulations


In [2]:
import os
import qiskit
import XW_waves.xw_1d as xw
mix = xw.mix
mse = xw.mse
reload = mix.reload_module
import pylib.circuit_lib as lib
os.getpid()

10503

In [5]:
# -------------------------------------------
# --- Create initial circuit (operator A) ---
# -------------------------------------------
a = qiskit.QuantumRegister(1, "a")
qc = qiskit.QuantumCircuit(a,name="A")

coef_v = 0.1
qc.ry(2*np.arccos(coef_v), a)

# set |1> as a good state:
qc.x(a)

# store the circuit as a gate:
gA = qc.to_gate()

print(qc.draw(fold = 110))

str_wv, pp, qq = lib.Wavefunction_adv(qc, column=True, systems=[1], width=10)
print(str_wv)
lib.analysis_prob(pp, qq)

   ┌────────────┐┌───┐
a: ┤ Ry(2.9413) ├┤ X ├
   └────────────┘└───┘
        0.995+0.000j |0>   
        0.100+0.000j |1>   

prob. of GS: 1.000e-02
prob. of BS: 9.900e-01
BS + GS: 1.000e+00
optimal number of amplification operators:  7.840854384487761


(0.010000000000000002, 0.9900051001000001)

In [6]:
# -----------------------------------------
# --- Amplitude amplification circuit ---
# -----------------------------------------

# Q = - A R_0 A^{-1} R_good;
# |good> = |1>

a = qiskit.QuantumRegister(1, "a")

qc = qiskit.QuantumCircuit(a,name="AA")

# sign change of good state (state |1>)
qc.z(a)

# inverse creation gate
gAi = gA.inverse()
qc.append(gAi, a)

# sign change of zero state 
qc.x(a)
qc.z(a)
qc.x(a)

# the creation gate again:
qc.append(gA, a)

print(qc.draw())

gAA = qc.to_gate()

   ┌───┐┌──────┐┌───┐┌───┐┌───┐┌───┐
a: ┤ Z ├┤ A_dg ├┤ X ├┤ Z ├┤ X ├┤ A ├
   └───┘└──────┘└───┘└───┘└───┘└───┘


In [8]:
# ---------------------------------------
# --- The whole circuit ---
# ---------------------------------------
a = qiskit.QuantumRegister(1, "a")

qc = qiskit.QuantumCircuit(a,name="AA")
qc.append(gA, a)

print("--- before amplification ---")
print(qc.draw(fold = 110))
str_wv, pp, qq = lib.Wavefunction_adv(qc, column=True, systems=[1], width=10)
print(str_wv)
lib.analysis_prob(pp, qq)

# -------------------------------------------------------------------------------
# --- Several amplifications at one ---
na = 8 # number of amplitifications

print("\n\n--- after the {:d}-th amplifications ---".format(na))
for ii in range(na):
    qc.append(gAA, a)
print(qc.draw(fold = 110))
str_wv, pp, qq = lib.Wavefunction_adv(qc, column=True, systems=[1], width=10)
print(str_wv)
lib.analysis_prob(pp, qq)


# # -------------------------------------------------------------------------------
# # --- Amplificaitions: one by one ---
# print("\n\n--- after the 1-st amplification ---")
# qc.append(gAA, a)
# print(qc.draw(fold = 110))
# str_wv, pp, qq = lib.Wavefunction_adv(qc, column=True, systems=[1], width=10)
# print(str_wv)
# lib.analysis_prob(pp, qq)

# print("\n\n--- after the 2-nd amplification ---")
# qc.append(gAA, a)
# print(qc.draw(fold = 110))
# str_wv, pp, qq = lib.Wavefunction_adv(qc, column=True, systems=[1], width=10)
# print(str_wv)
# lib.analysis_prob(pp, qq)

# print("\n\n--- after the 3-rd amplification ---")
# qc.append(gAA, a)
# print(qc.draw(fold = 110))
# str_wv, pp, qq = lib.Wavefunction_adv(qc, column=True, systems=[1], width=10)
# print(str_wv)
# lib.analysis_prob(pp, qq)

--- before amplification ---
   ┌───┐
a: ┤ A ├
   └───┘
        0.995+0.000j |0>   
        0.100+0.000j |1>   

prob. of GS: 1.000e-02
prob. of BS: 9.900e-01
BS + GS: 1.000e+00
optimal number of amplification operators:  7.840854384487761


--- after the 8-th amplifications ---
   ┌───┐┌────┐┌────┐┌────┐┌────┐┌────┐┌────┐┌────┐┌────┐
a: ┤ A ├┤ AA ├┤ AA ├┤ AA ├┤ AA ├┤ AA ├┤ AA ├┤ AA ├┤ AA ├
   └───┘└────┘└────┘└────┘└────┘└────┘└────┘└────┘└────┘
       -0.132+0.000j |0>   
        0.991+0.000j |1>   

prob. of GS: 9.827e-01
prob. of BS: 1.734e-02
BS + GS: 1.000e+00
optimal number of amplification operators:  0.5459023467761799


(0.9826558641, 0.017336988900000003)